## Installation und Einrichtung
Bevor du startest, stelle sicher, dass du die folgenden Pakete installiert hast. Führe in einer Jupyter-Notebook-Zelle Folgendes aus:

``pip install stable-baselines3 pandas matplotlib ccxt``

stable-baselines3: Für Reinforcement Learning.
pandas: Für die Datenverarbeitung.
matplotlib: Für die grafische Darstellung.
ccxt: Für die API-Anbindung an Kryptobörsen wie Binance.

##  Datenimport und Vorbereitung
In dieser Zelle holen wir die historischen Bitcoin-Daten:

In [ ]:
import yfinance as yf

# API-Anbindung und Datenabruf mit Yahoo Finance
def fetch_bitcoin_data(start_date, end_date):
    """
    Ruft Bitcoin-Daten von Yahoo Finance für den angegebenen Zeitraum ab.

    Args:
        start_date (str): Startdatum im Format 'YYYY-MM-DD'.
        end_date (str): Enddatum im Format 'YYYY-MM-DD'.

    Returns:
        pd.DataFrame: DataFrame mit den historischen Kursdaten.
    """
    # Lade Daten von Yahoo Finance
    df = yf.download("BTC-USD", start=start_date, end=end_date, interval="1d")

    # Umbenennen der Spalten für Konsistenz
    df.rename(columns={
        "Open": "open",
        "High": "high",
        "Low": "low",
        "Close": "close",
        "Volume": "volume"
    }, inplace=True)

    # Filtere die relevanten Spalten und setze den Zeitstempel als Index
    df = df[["open", "high", "low", "close", "volume"]]
    df.index.name = "timestamp"

    return df



In [2]:
# Abrufen der Daten
data = fetch_bitcoin_data('2018-01-01', '2023-12-01')
print(data.head())  # Zeige die ersten Zeilen der Daten

[*********************100%***********************]  1 of 1 completed

Price               open          high           low         close  \
Ticker           BTC-USD       BTC-USD       BTC-USD       BTC-USD   
timestamp                                                            
2018-01-01  14112.200195  14112.200195  13154.700195  13657.200195   
2018-01-02  13625.000000  15444.599609  13163.599609  14982.099609   
2018-01-03  14978.200195  15572.799805  14844.500000  15201.000000   
2018-01-04  15270.700195  15739.700195  14522.200195  15599.200195   
2018-01-05  15477.200195  17705.199219  15202.799805  17429.500000   

Price            volume  
Ticker          BTC-USD  
timestamp                
2018-01-01  10291200000  
2018-01-02  16846600192  
2018-01-03  16871900160  
2018-01-04  21783199744  
2018-01-05  23840899072  


- **fetch_ohlcv**: Ruft Open, High, Low, Close, Volume (OHLCV) Daten ab.
- **timestamp**: Zeitstempel für jeden Tag.
- **close**: Schlusskurs, der für das Training verwendet wird

## Umgebung und Agent erstellen
Hier definieren wir die RL-Umgebung und den Agenten:

In [3]:
import gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv  # Korrigierter Import

# Custom Environment für RL
class BitcoinTradingEnv(gym.Env):
    def __init__(self, data, window_size=30):
        super(BitcoinTradingEnv, self).__init__()
        self.data = data['close'].values
        self.window_size = window_size
        self.current_step = window_size

        # Action Space: Kaufen, Verkaufen, Halten
        self.action_space = gym.spaces.Discrete(3)

        # Observation Space: Preisfenster (normalisiert)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(window_size,), dtype=np.float32)

    def reset(self):
        self.current_step = self.window_size
        return self._get_observation()

    def _get_observation(self):
        """
        Gibt das Fenster der letzten `window_size` Tage als flaches Array zurück.
        """
        window = self.data[self.current_step - self.window_size:self.current_step]
        normalized_window = (window - np.min(window)) / (np.max(window) - np.min(window))

        # Konvertiere die Beobachtung in ein flaches Array (1D)
        return normalized_window.flatten()


    def step(self, action):
        reward = 0  # Default-Belohnung

        # Verhindere Indexüberschreitung
        if self.current_step >= len(self.data) - 2:
            done = True
            return self._get_observation(), reward, done, {}

        # Aktion auswerten
        if action == 1:  # Kaufen
            reward = (self.data[self.current_step + 1] - self.data[self.current_step])  # Gewinn bei Kursanstieg
        elif action == 2:  # Verkaufen
            reward = (self.data[self.current_step] - self.data[self.current_step + 1])  # Gewinn bei Kursrückgang
        elif action == 0:  # Halten
            reward = -abs(self.data[self.current_step + 1] - self.data[self.current_step])  # Verlust bei Halten

        # Schritt weiter
        self.current_step += 1
        done = self.current_step >= len(self.data) - 1

        return self._get_observation(), reward, done, {}


# Initialisiere die Umgebung
env = DummyVecEnv([lambda: BitcoinTradingEnv(data, window_size=60)])



c:\Users\tlfin\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [4]:
# Teste die Beobachtung
obs = env.reset()
print("Beobachtung nach Reset:", obs.shape)  # Sollte (1, 60) sein, da `DummyVecEnv` die Dimension erweitert

Beobachtung nach Reset: (1, 60)


## Training und Modell speichern
Führe diesen Code aus, um das Modell zu trainieren und zu speichern:

In [5]:
# Schritt 1: Daten für den Trainingszeitraum laden
data = fetch_bitcoin_data('2018-01-01', '2023-12-31')  # Ruft die Daten von 2018 bis Ende 2023 ab

# Schritt 2: Umgebung mit den Trainingsdaten initialisieren
env = DummyVecEnv([lambda: BitcoinTradingEnv(data, window_size=60)])  # 60-Tage-Fenster

# Schritt 3: Modell laden oder neu erstellen
try:
    model = PPO.load("bitcoin_trading_model.zip", env=env)  # Versuche, das Modell zu laden
    print("Modell geladen.")
except FileNotFoundError:
    model = PPO('MlpPolicy', env, verbose=1)  # Erstelle ein neues Modell, falls keins existiert
    print("Neues Modell erstellt.")

# Schritt 4: Trainiere das Modell
model.learn(total_timesteps=50000)

# Schritt 5: Speichere das Modell
model.save("bitcoin_trading_model.zip")

[*********************100%***********************]  1 of 1 completed
c:\Users\tlfin\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Modell geladen.
-----------------------------
| time/              |      |
|    fps             | 1411 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 944         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 2.36099e-06 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.505      |
|    explained_variance   | 0.019367099 |
|    learning_rate        | 0.0003      |
|    loss                 | 4.21e+06    |
|    n_updates            | 760         |
|    policy_gradient_loss | -1.66e-05   |
|    value_loss           | 1.01e+07    |
-----------------------------------------
------------------